In [ ]:
#!pip install einops

In [ ]:
import os
import gc
import torch
import wandb
import pytorch_lightning as pl
import torch.nn.functional as F


from torch import nn
from typing import *
from einops import rearrange
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from pytorch_lightning.tuner.tuning import Tuner
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint


torch.random.manual_seed(0)
pl.seed_everything(0)

# Patch embeddings

In [ ]:
class PatchEmbedding(nn.Module):
    """ 
    Image to Patch Embedding
    """
    def __init__(
        self,
        img_size: int = 224,
        patch_size: int = 16,
        in_chans: int = 3,
        d_model: int = 768
    ):
        super().__init__()
        
        self.d_model = d_model
        self.in_chans = in_chans
        self.img_size = img_size
        
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_embeddings = nn.Conv2d(3, self.d_model, patch_size, patch_size)

    def forward(self, image):
        b, c, h, w = image.shape
        
        assert h == self.img_size and w == self.img_size, f'Image size must be {self.img_size}x{self.img_size}'
        assert c == self.in_chans, f'Image must have {self.in_chans} channels'
        
        patches = self.patch_embeddings(image).reshape(b, self.d_model, -1).transpose(1, 2)
        
        return patches

In [ ]:
x = torch.randn((2, 3, 224, 224))
PatchEmbedding()(x).shape

# Residual block

In [ ]:
class ResidualBlock(nn.Module):
    
    def __init__(self, func: Optional[Callable] = None) -> None:
        super().__init__()
        
        self.func = func
        if not self.func:
            self.func = lambda x: x
    
    def forward(self, x):
        x = self.func(x) + x
        return x

In [ ]:
x = torch.Tensor([1., 2., 3., 4.])
ResidualBlock(lambda x: x**2)(x)

# Multi Head Attention Block

In [ ]:
class MHABlock(nn.Module):
    def __init__(
        self,
        emb_len: int,
        num_heads: int = 8,
        attn_drop: float = 0.,
        out_drop: float = 0.
    ):
        super().__init__()
        
        self.num_heads = num_heads # number of heads
        head_emb = emb_len // num_heads # embeddings length after head
        self.scale = head_emb ** -0.5 # scale param for decrease dispersion

        self.qkv = nn.Linear(emb_len, emb_len * 3, bias=False)
        self.attn_drop = nn.Dropout(attn_drop)
        
        self.out = nn.Sequential(
            nn.Linear(emb_len, emb_len),
            nn.Dropout(out_drop)
        )
        

    def forward(self, x):
        
        QKV = self.qkv(x)
        """
        b - batch
        l - sequence length (number of patches)
        n - 3 (Q K V)
        h - num heads
        hl - seq length after attention
        """
        Q, K, V = rearrange(QKV, 'b l (n h hl) -> n b h l hl', n = 3, h = self.num_heads)

        attention = F.softmax(torch.einsum('bhqo, bhko -> bhqk', Q, K) / self.scale, dim=-1)
        attention = self.attn_drop(attention)
        attention = attention @ V
        attention = rearrange(attention, 'b h l hl -> b l (h hl)')
        
        out = self.out(attention)
        return out


In [ ]:
x = torch.randn((5, 197, 768))
MHABlock(768)(x).shape

# Feed forward block

In [ ]:
class FeedForwardBlock(nn.Module):
    def __init__(
        self, 
        in_features: int, 
        mlp_ratio: int = 4,
        hidden_features: Optional[int] = None, 
        out_features: Optional[int] = None, 
        drop_rate: float = 0.
    ):
        super().__init__()
        
        if not hidden_features:
            hidden_features = in_features * mlp_ratio
        if not out_features:
            out_features = in_features

        self.linears = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.GELU(),
            nn.Dropout(drop_rate),
            nn.Linear(hidden_features, out_features),
        )

    def forward(self, x):
        x = self.linears(x)
        return x

In [ ]:
x = torch.randn(1, 197, 768)
FeedForwardBlock(768)(x).shape

# Encoder block

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(
        self, 
        emb_len: int, 
        num_heads: int = 8, 
        mlp_ratio: int = 4, 
        drop_rate: float =0.
    ):
        super().__init__()

        self.first_residual = ResidualBlock(
            nn.Sequential(
                nn.LayerNorm(emb_len),
                MHABlock(emb_len, num_heads, drop_rate, drop_rate),
                nn.Dropout(drop_rate)
            )
        )
        
        self.second_residual = ResidualBlock(
            nn.Sequential(
                nn.LayerNorm(emb_len),
                FeedForwardBlock(emb_len, mlp_ratio),
                nn.Dropout(drop_rate)
            )
        )           

    def forward(self, x):
        
        x = self.first_residual(x)
        x = self.second_residual(x)
        
        return x

In [ ]:
x = torch.randn(1, 197, 768)
block = EncoderBlock(768, 12)
out = block(x)
out.shape

# Transformer class. Stack of EncoderBlocks

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self, 
        num_layers: int, 
        emb_len: int, 
        num_heads: int = 12,
        mlp_ratio: int = 4,
        drop_rate: float = 0.
    ):
        super().__init__()
        self.blocks = nn.ModuleList([
            EncoderBlock(emb_len, num_heads, mlp_ratio, drop_rate)
            for i in range(num_layers)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x

In [ ]:
x = torch.randn(1, 197, 768)
Transformer(12, 768)(x).shape

# Vision Transformer model

In [ ]:
class ViT(pl.LightningModule):
  
    def __init__(
        self,
        img_size: int = 224,
        patch_size: int = 16,
        in_chans: int = 3,
        num_classes: int = 1000,
        emb_len: int = 768,
        num_layers: int = 12,
        num_heads: int = 12,
        mlp_ratio: int = 4,
        drop_rate: int = 0.,
        loss_func = None
    ):
        super(ViT, self).__init__()
        self.save_hyperparameters(ignore=['loss_func'])
        
        # Path Embeddings, CLS Token, Position Encoding
        self.patch_embeddings = PatchEmbedding(img_size, patch_size, in_chans, emb_len)
        self.cls_token = nn.Parameter(torch.randn((1, 1, emb_len)))
        self.pos_encodings = nn.Parameter(torch.randn((self.patch_embeddings.num_patches + 1, emb_len)))

        # Transformer Encoder
        self.transformer = Transformer(num_layers, emb_len, num_heads, mlp_ratio, drop_rate)

        # Classifier
        self.classifier = nn.Linear(emb_len, num_classes)
        
        self.loss_func = loss_func


    def forward(self, x):
        # Path Embeddings, CLS Token, Position Encoding
        b, c, h, w = x.shape
        
        cls_tokens = self.cls_token.expand(b, -1, -1)
        x = self.pos_encodings + torch.cat((cls_tokens, self.patch_embeddings(x)), dim = 1)

        # Transformer Encoder
        x = self.transformer(x)[:, 0, :].squeeze(1)

        # Classifier
        predictions = self.classifier(x)

        return predictions


    # Настраиваются параметры обучения
    def training_step(self, batch, batch_idx):
        data, targets = batch
        
        logits = self(data)
        
        loss = self.loss_func(logits, targets)
        accuracy = torch.sum(logits.argmax(-1) == targets) / len(logits)
        
        lr = self.lr_schedulers().get_last_lr()[-1]
        self.log('loss', loss, on_epoch=True, on_step=False)
        self.log('acc', accuracy, on_epoch=True, on_step=False)
        self.log('Lr', lr, on_epoch=True, on_step=False)
        
        output = {
            'loss': loss,
            'acc': accuracy,
            'lr': lr
        }
        
        return output


    # Настраиваются параметры тестирования
    def test_step(self, batch, batch_idx):
        data, targets = batch
        logits = self(data)
        
        loss = self.loss_func(logits, targets)
        accuracy = torch.sum(logits.argmax(-1) == targets) / len(logits)
    
        self.log('Test acc', accuracy, prog_bar=True) 
        output = {
            'loss': loss,
            'acc': accuracy
        }
        return output


    # Конфигурируется оптимизатор
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer, 
            max_lr = 0.0004,
            total_steps = self.trainer.max_epochs * self.trainer.datamodule.len_train_dataloader,
            pct_start = 0.05)
        
        config = {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'step'
            }
        }
        
        return config
    
    def training_epoch_end(self, outputs) -> None:
        loss = sum(output['loss'] for output in outputs) / len(outputs)
        print(f'Эпоха {self.current_epoch}, loss = {loss}')

In [ ]:
x = torch.randn(10, 3, 224, 224)
vit = ViT(num_classes=15)
out = vit(x)
out.shape

# Data

In [ ]:
class DataModule(pl.LightningDataModule):
    
    
    def __init__(
        self,
        root_dir: str,
        train_folder: str,
        test_folder: str,
        batch_size: int
    ) -> None:
        super(pl.LightningDataModule, self).__init__()
        
        self.train_dir = os.path.join(root_dir, train_folder)
        self.test_dir = os.path.join(root_dir, test_folder)
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)), 
            transforms.ToTensor()])
        
        self.batch_size = batch_size
    
    
    def setup(self, stage: Optional[str] = None) -> None:
        if stage == 'fit':
            self.train_data = datasets.ImageFolder(self.train_dir, self.transform)
            self.len_train_dataloader = len(self.train_data) // self.batch_size
        if stage == 'test':
            self.test_data = datasets.ImageFolder(self.test_dir, self.transform)
    
    
    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_data, self.batch_size, True, drop_last=True)

    
    def test_dataloader(self) -> DataLoader:
        return DataLoader(self.test_data, self.batch_size, False, drop_last=True)

# Config

In [ ]:
patch_size = 16
num_heads = 8
num_layers = 8
emb_len = 384
drop_rate = 0.07
max_lr = 0.0004
epochs = 200
root_dir = '../input/vegetable-image-dataset/Vegetable Images'

# Init modules

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test acc'
        },
    'parameters': {
        'patch_size': {'values': [14, 16]},
        'num_heads': {'values': [8, 12]}
     }
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project = "First ViT")

In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config
        run.name = f"P_{config.patch_size} \
                     -H_{config.num_heads} \
                     -L_{num_layers} \
                     -E_{emb_len} \
                     -D_{drop_rate} \
                     -LR_{max_lr} \
                     -Epochs_{epochs}"

        gc.collect()
        torch.cuda.empty_cache()

        try:
            pl.seed_everything(seed=0)
            
            print('Initializing modules...')
            
            datamodule = DataModule(root_dir, 'validation', 'test', 64)
            model = ViT(
                patch_size=config.patch_size,
                num_heads=config.num_heads,
                num_layers=num_layers,
                emb_len=emb_len,
                num_classes=15,
                drop_rate=drop_rate,
                loss_func=nn.CrossEntropyLoss()
            )
            wandb_logger = WandbLogger(log_model = True)
            wandb_logger.watch(model, log = 'all', log_freq = 100)
            trainer = pl.Trainer(
                accelerator = 'gpu',
                max_epochs = epochs,
                default_root_dir = './lightning',
                logger = wandb_logger
            )
            
            print('Initializing successful...')
            
            print('Start training')
            trainer.fit(model, datamodule)
            print('Training is finished')
            
            print('Start testing')
            trainer.test(model, datamodule)
            print('Testing is finished')
            
        except Exception as e:
            print(e)

        del model
        del datamodule
        del wandb_logger
        del trainer
            
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
wandb.agent(sweep_id, function=train, count=10)

![Image-alt](./Sweeps.jpg)